In [1]:
import networkx as nx
import numpy as np
import itertools
from scipy.sparse import csr_matrix

# 创建一个无向图
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4, 5])
G.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 5)])

# 构建邻接矩阵
A = nx.adjacency_matrix(G)
n = A.shape[0]

# 将邻接矩阵转换为稀疏矩阵
A = csr_matrix(A)

# 定义目标函数
def objective_function(x):
    return x.dot(A.dot(1 - x))

# 枚举所有可能的割集
max_cut_value = 0
max_cut = None
for i in range(1, n // 2 + 1):
    for subset in itertools.combinations(range(n), i):
        x = np.zeros(n)
        x[list(subset)] = 1
        cut_value = objective_function(x)
        if cut_value > max_cut_value:
            max_cut_value = cut_value
            max_cut = subset

print("最大割集：", max_cut)
print("最大割值：", max_cut_value)

最大割集： (1, 3)
最大割值： 4.0


C:\Users\Newt\AppData\Local\Temp\ipykernel_14308\2542940655.py:12: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)


In [4]:
# 创建无向图
graph = {
    0: [1, 2],
    1: [0, 2, 3],
    2: [0, 1, 4],
    3: [1, 4],
    4: [2, 3]
}

# 初始化残余网络
def init_residual_network(graph):
    residual_network = {}
    for node in graph:
        residual_network[node] = {}
        for neighbor in graph[node]:
            residual_network[node][neighbor] = graph[node].count(neighbor)
    return residual_network

# 寻找增广路径
def find_augmenting_path(residual_network, source, sink, path):
    if source == sink:
        return True
    for neighbor in residual_network[source]:
        if neighbor not in path and residual_network[source][neighbor] > 0:
            path.append(neighbor)
            if find_augmenting_path(residual_network, neighbor, sink, path):
                return True
            path.pop()
    return False

# Ford-Fulkerson算法
def ford_fulkerson(graph, source, sink):
    residual_network = init_residual_network(graph)
    max_flow = 0
    while find_augmenting_path(residual_network, source, sink, []):
        path_flow = float('inf')
        s = sink
        while s != source:
            u = path.pop()
            v = path.pop()
            path_flow = min(path_flow, residual_network[u][v])
            s = u
        max_flow += path_flow
        v = sink
        while v != source:
            u = path.pop()
            residual_network[u][v] -= path_flow
            residual_network[v][u] += path_flow
            v = u
    return max_flow

# 计算最大割
def max_cut(graph, source, sink):
    max_flow = ford_fulkerson(graph, source, sink)
    visited = set()
    path = []
    dfs(graph, source, sink, visited, path)
    return max_flow, [node for node in graph if node not in visited]

# DFS遍历图
def dfs(graph, node, sink, visited, path):
    visited.add(node)
    path.append(node)
    if node == sink:
        return True
    for neighbor in graph[node]:
        if neighbor not in visited and graph[node][neighbor] > 0:
            if dfs(graph, neighbor, sink, visited, path):
                return True
    path.pop()
    return False

# 测试
source = 0
sink = 4
print("最大割为：", max_cut(graph, source, sink))


NameError: name 'path' is not defined